# (4.1) Train a state-action value function (Q-function) on an AFA dataset
In this notebook we train a Q-function that estimates the expected cost to go if we were in a fully observed environment. 

In [1]:
%load_ext autoreload
%autoreload 2

## Define paths

Paths for data

In [2]:
from afa.configurations.utils_ts import specify_default_paths_ts
# which dataset to work on 
dataset_name   = "synthetic_1"

# name for of missingness scenario 
miss_scenario  = 'MCAR_1'

# automatically specify some path locations (change paths manually if needed) 
paths = specify_default_paths_ts(dataset_name = dataset_name , miss_scenario = miss_scenario) 

# name for agent 
agent_name            = 'Random 10%' #'DQN' #'Random 10%' # 'DQN' #'Random 10%' # 'DQN' ##'Random 10%'
agent_dir = paths['data_dir'] + 'afa_agents' + '/' + agent_name + '/'

# how to name the afa_dataset
afa_dataset_name = 'blocking'

In [3]:
# name for predictor 
q_model_name  = 'q_model' #'logistic_regression'

# new (where to save the model) 
q_model_dir = paths['data_dir'] + 'q_models' + '/' + q_model_name + '/'

# reporting
explanation_file = q_model_dir +  'reports/' + 'model_report' 

## Load afa dataset

In [4]:
from afa.data_modelling.datasets.data_loader.data_loader_ts import DataLoader_ts
from afa.afa_datasets.afa_data_loader.afa_data_loader_ts import AFADataLoader_ts

2023-04-27 16:45:55.751947: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 16:45:56.117801: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-27 16:45:56.117846: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-27 16:45:57.522162: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [5]:
# load dataset 
data_loader = DataLoader_ts(     data_file                  = paths['data_file'],
                                 temporal_data_file         = paths['temporal_data_file'],
                                 superfeature_mapping_file  = paths['superfeature_mapping_file'],
                                 problem_file               = paths['problem_file'],
                                 afa_problem_files          = paths['afa_problem_files'], 
                                 miss_model_files           = paths['miss_model_files'], 
                                 folds_file                 = paths['folds_file'] )
dataset = data_loader.load() 

Padding sequences: 100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 855.19it/s]


In [6]:
# load afa_dataset
augmented_data_file = agent_dir + afa_dataset_name + '_' + 'results.hkl'
afa_agent_params = None
afa_data_loader = AFADataLoader_ts(                   
                    augmented_data_file = augmented_data_file,
                    dataset             = dataset,
                    model_params        = afa_agent_params) 
afa_dataset = afa_data_loader.load() 

2023-04-27 16:46:25.504106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-27 16:46:25.504516: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-04-27 16:46:25.504638: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2023-04-27 16:46:25.504733: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2023-04-27 16:46:25.504826: W tensorflow/co

> /mnt/c/Users/henrik.vonkleist/Nextcloud/PhD/Code/Active Feature Acquisition/afa_ts/afa/afa_models/afa_nuisance_models/afa_mechanism_nuisance_dataset_ts.py(47)__init__()
     45         pdb.set_trace()
     46 
---> 47         return
     48 
     49     def _compute_afa_ipw_weights(self, R_bar_samples):



ipdb>  self.R_hat_noblocking.shape


(200, 5, 2, 1)


ipdb>  exit


## Initialize Q function 

In [ ]:
from afa.afa_models.afa_nuisance_models.afa_q_models.afa_q_model_ts import AFAQModel_ts

In [ ]:
q_model_params = {
    'name' : q_model_name, 
    'directory': q_model_dir,
    'base_model_params' : {   
        'hidden_size': 16,
        'epochs' :   10,
        'batch_size' : 32
    }
}

In [ ]:
afa_q_model = AFAQModel_ts( name            = q_model_params['name'], 
                            model_params    = q_model_params, 
                            afa_dataset     = afa_dataset, 
                            directory       = q_model_params['directory'])   

In [ ]:
afa_q_model.fit(  afa_dataset, 
                  fold = 0, 
                  train_split = 'train_afa_nuisance', 
                  valid_split = 'test', 
                  fit_again = False)

In [ ]:
q_values, v_values  = afa_q_model.predict(   afa_dataset, 
                                              fold = None, 
                                              split = None)

In [ ]:
# add to afa_dataset 
results = {}
results['q_values_' + q_model_name] = q_values
results['v_values_' + q_model_name] = v_values

In [ ]:
afa_dataset.set_augmented_data( 
                            results,
                            set_nuisances = True, 
                            )

In [ ]:
afa_dataset.results['nuisances'].keys()

In [ ]:
data, results_new = afa_dataset.get_augmented_data(get_nuisances = True)
results_new['nuisances'].keys()

## Save afa_dataset with Q and V values

In [ ]:
# save afa_dataset
afa_dataset.save( directory = agent_dir, afa_dataset_name = afa_dataset_name ) 

## Test loading afa dataset with Q and V values

In [ ]:
from afa.afa_datasets.afa_data_loader.afa_data_loader_static import AFADataLoader_static

In [ ]:
augmented_data_file = agent_dir + afa_dataset_name + '_' + 'results.hkl'
afa_data_loader = AFADataLoader_static(                   
                    augmented_data_file = augmented_data_file,
                    dataset  = dataset,
                    model_params = afa_agent_params) 
afa_dataset = afa_data_loader.load() 

In [ ]:
data, results_new = afa_dataset.get_augmented_data(get_nuisances = True)
results_new['nuisances'].keys()

### Write report

In [ ]:
# report 
# explanation_file = q_model_dir + afa_dataset_name + '_' + 'q_values_report.md'  # +  'reports/' + 'model_report' 
# afa_dataset.explain(file= explanation_file, format = 'markdown')